In [1]:
! nvidia-smi

Fri Mar 18 00:11:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 950M    On   | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8    N/A /  N/A |      7MiB /  2004MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ! pip install navec
# ! pip install slovnet

# ! pip install -U pip setuptools wheel
# ! pip install -U spacy
# ! python -m spacy download en_core_web_sm
# ! python -m spacy download ru_core_news_sm

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# % cd drive/MyDrive/HSE/NLP/stihoplet/
# !ls

In [4]:
# ! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
%matplotlib inline

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import torch
from torch import nn
import torchvision.models as models
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, Subset
from torch.utils.data.dataset import T_co
from torchvision import transforms

import spacy
nlp = spacy.load("ru_core_news_sm")

from slovnet.model.emb import NavecEmbedding
from navec import Navec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

[nltk_data] Downloading package punkt to /home/sergey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
SEED = 0
np.random.seed(SEED)
torch.manual_seed(SEED)

In [7]:
# df_raw = pd.read_csv("poems_2021_love.csv", header=None, usecols=[4])
# df = pd.read_csv("clean_lemma_reversed.csv", header=None, usecols=[4, 'clean_reversed'])
df = pd.read_csv("data/clean_lemma_reversed.csv", index_col=0)
df.head()

,4,clean_reversed
0,"Так хочется, чтоб кто-нибудь обнял.\nИ просто ...",обнять чтоб хотеться так\nночь спокойной пожел...
1,"Когда я встретил, помню, Вас\nВ морозный вечер...",вас помнить встретить я когда\nпарк зимнем в в...
2,Е.З.\n\nМир соткан из противоречий\nТвоих молч...,противоречие из соткан мир\nшаг мой и молчание...
3,Я тебя никогда не забуду.\nА ты мне никогда не...,забыть не никогда тебя я\nответишь не никогда ...
4,Вы видели влюблённые глаза?\nКоторые так смотр...,глаз влюблённые видеть вы\nинтересно смотреть ...


In [8]:
def preprocess(text, lemma=False):
    if not text:
        return ""
    new_text = []
    lines = str(text).split('\n')
    for line in lines:
        words = " ".join(reversed([word.lower() for word in word_tokenize(str(line)) if word.isalpha()]))
        if len(words) < 1:
            continue
        if lemma:
            words = " ".join([token.lemma_ for token in nlp(words)])
        new_text.append(words)
    return "\n".join(new_text)

In [9]:
%%time
#df['clean_reversed'] = df[4].apply(lambda x: preprocess(x, lemma=True))

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 18.8 µs


In [10]:
#df.to_csv("clean_lemma_reversed.csv")
df.head()

,4,clean_reversed
0,"Так хочется, чтоб кто-нибудь обнял.\nИ просто ...",обнять чтоб хотеться так\nночь спокойной пожел...
1,"Когда я встретил, помню, Вас\nВ морозный вечер...",вас помнить встретить я когда\nпарк зимнем в в...
2,Е.З.\n\nМир соткан из противоречий\nТвоих молч...,противоречие из соткан мир\nшаг мой и молчание...
3,Я тебя никогда не забуду.\nА ты мне никогда не...,забыть не никогда тебя я\nответишь не никогда ...
4,Вы видели влюблённые глаза?\nКоторые так смотр...,глаз влюблённые видеть вы\nинтересно смотреть ...


In [4]:
import torch
import torchtext
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [5]:
from collections import Counter, defaultdict

In [6]:
df_sentences = pd.read_csv("data/poems_2021_love_lemma_sentences.csv", index_col=0)
df_sentences.head()

,sentence
0,так хотеться чтоб обнять
1,и просто пожелать спокойной ночь
2,и уходить чтоб в щека целовать
3,желать вновь увидеть очень очень
4,на расстояние думать обо мне


In [7]:
def get_vocab(sentences):
    s = set()
    for sentence in sentences:
        s.update(sentence.split(" "))
    
    vocab = defaultdict(lambda: 0)
    vocabr = defaultdict(lambda: "<unk>")
    vocab["<unk>"] = 0
    vocabr[0] = "<unk>"
    vocab["<pad>"] = 1
    vocabr[1] = "<pad>"
    idx = 2

    for word in s:
        vocab[word] = idx
        vocabr[idx] = word
        idx += 1
    
    return vocab, vocabr

In [8]:
def tokenize(sentences, vocab, max_length=10):
    data = list()
    for sentence in sentences:
        token_list = [vocab[word] for word in sentence.split(' ')][:max_length]
        for i in range(1, len(token_list)):
            n_gram_seq = torch.tensor(token_list[:i+1], dtype=torch.long)
            data.append(n_gram_seq)
    return data

In [9]:
%%time
# sentences = ['я ты он птицеед она птица', 'я помню чудное мгновенье']
MAX_LENGTH = 10
vocab, vocabr = get_vocab(df_sentences['sentence'])
train_data = tokenize(df_sentences['sentence'], vocab, max_length=MAX_LENGTH)
print(len(train_data), len(vocab), train_data[0])

763702 68152 tensor([28750, 46067])
CPU times: user 4.17 s, sys: 156 ms, total: 4.33 s
Wall time: 4.34 s


In [10]:
# train_data = train_data[:100000]

In [11]:
%%time
X_train = [i[:-1] for i in train_data]   # taking all the words except the last in the input set
y_train = [i[-1] for i in train_data] 

X_train = pad_sequence(X_train, batch_first=True, padding_value=vocab['<pad>'])   # padding the text_seq so the training sample are of equal length
y_train = torch.from_numpy(np.array(y_train))
print(X_train.shape, X_train[3])

torch.Size([763702, 9]) tensor([2908,    1,    1,    1,    1,    1,    1,    1,    1])
CPU times: user 8.14 s, sys: 488 ms, total: 8.63 s
Wall time: 9.64 s


In [12]:
train_data = TensorDataset(X_train, y_train)

In [13]:
BATCH_SIZE = 128
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [15]:
class PoetryLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout=0.15):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, X, h=None, c=None):
        if h is None:
            h, c = self.init_state(X.size(0))
        out = self.embedding(X)
        out, (h, c) = self.lstm(out, (h, c))
        out = out.contiguous().view(-1, self.hidden_size)
        out = self.fc1(out)
        out = out.view(-1, X.size(1), self.vocab_size)
        out = out[:, -1]
        
        return out, h, c
    
    def init_state(self, batch_size):
        num_l = self.num_layers
        hidden = torch.zeros(num_l, batch_size, self.hidden_size).to(DEVICE)
        cell = torch.zeros(num_l, batch_size, self.hidden_size).to(DEVICE)
        return hidden, cell

In [16]:
VOCAB_SIZE = len(vocab) # 500002 # len(navec.vocab.top())
EMBEDDING_DIM = 128
HIDDEN_SIZE = 128
NUM_LAYERS = 3

print(VOCAB_SIZE) # 68152

68152


In [17]:
%%time
model = PoetryLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE, NUM_LAYERS).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

CPU times: user 3.27 s, sys: 718 ms, total: 3.99 s
Wall time: 24.5 s


In [25]:
# train loop
# EPOCHS = 60

# for epoch in tqdm(range(EPOCHS)):
#     epoch_loss = 0
#     for X, y in tqdm(train_dataloader):
#         X = X.to(DEVICE)
#         y = y.to(DEVICE)
        
#         optimizer.zero_grad()
#         output, h, c = model(X)
#         loss = criterion(output, y)
#         epoch_loss += loss
#         loss.backward()
#         nn.utils.clip_grad_norm_(model.parameters(), 5) # Clipping Gradients
#         optimizer.step()
    
#     if (epoch + 1) % 5 == 0:
#         torch.save(model.state_dict(), f"models/{epoch+1}ep_128x128x3.pth")
        
#     print(f"Epoch: {epoch+1} Loss:{epoch_loss/len(train_dataloader)}")

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/5967 [00:00<?, ?it/s]

Epoch: 1 Loss:7.884281158447266


  0%|          | 0/5967 [00:00<?, ?it/s]

Epoch: 2 Loss:7.3345232009887695


...

Epoch: 58 Loss:4.342421054840088


  0%|          | 0/5967 [00:00<?, ?it/s]

Epoch: 59 Loss:4.332004070281982


  0%|          | 0/5967 [00:00<?, ?it/s]

Epoch: 60 Loss:4.319840908050537


In [26]:
# torch.save(model.state_dict(), f"second_try_{EPOCHS}ep_128x128x3.pth")

In [18]:
model.load_state_dict(torch.load("models/lstm_lemma_60ep_128x128x3.pth"))
model.eval()

PoetryLSTM(
  (embedding): Embedding(68152, 128)
  (lstm): LSTM(128, 128, num_layers=3, batch_first=True, dropout=0.15)
  (fc1): Linear(in_features=128, out_features=68152, bias=True)
)

In [20]:
seed_text = "я тебя любить" #Starting of a song
next_words = 50

for i in range(next_words):
    token_list = np.ones(21, dtype=int)
    text_token = np.array([vocab[word] for word in seed_text.split(" ")][-MAX_LENGTH:])
    token_list[:len(text_token)] = text_token
    token_list = torch.from_numpy(token_list).unsqueeze(0).to(DEVICE)
    
    
    out,h,c = model(token_list)
    
    
    idx = int(torch.argmax(out))
    new_word = vocabr[idx]# if idx in vocabr else "<unk>"
    seed_text += " " + new_word
    
for i,word in enumerate(seed_text.split()):
    print(word,end=" "),
    if i!=0 and (i+1)%5==0:
        print("\n")

я тебя любить непристойность эпопея 

раскачивать легонечко возьмёте непристойность распрощайся 

расстанутся сосед взаимны вителю краснея 

утеряли свести непристойность вонзал надой 

пытанья окутают туманов пятен осяяння 

печатать надой железно смелым заманчив 

демократия лишний вонзал ношей оглушать 

joy вонзал надой искрились переноска 

песками убыток краснея чуять расстанутся 

порога раскачивать стаей замкнутому вдвоём 

локдаун глупышка спотыкаться 

In [28]:
seed_text = "так хотеться чтоб обнять" #Starting of a song
next_words = 50

for i in range(next_words):
    token_list = np.ones(21, dtype=int)
    text_token = np.array([vocab[word] for word in seed_text.split(" ")][-MAX_LENGTH:])
    token_list[:len(text_token)] = text_token
    token_list = torch.from_numpy(token_list).unsqueeze(0).to(DEVICE)
    
    
    out,h,c = model(token_list)
    
    
    idx = int(torch.argmax(out))
    new_word = vocabr[idx]# if idx in vocabr else "<unk>"
    seed_text += " " + new_word
    
for i,word in enumerate(seed_text.split()):
    print(word,end=" "),
    if i!=0 and (i+1)%5==0:
        print("\n")

так хотеться чтоб обнять меня 

капельками атом глаз внутри программа 

сигарета синтоми дама и тело 

из глагол африкой любименький в 

камне приземленно коготь натянуть чёрной 

акварели струить созерцать колесниченко остального 

мне не ворота бокал тепле 

быстро реалия за грех волнительный 

пусть бичурино портовый в прошлом 

сходу я в жизнь взглянуть 

и провозглашать трудность в 